In [220]:
from typing import List, Dict, Optional
from typing import List, Dict, Any
import json

In [221]:
class Prompt():
    def __init__(self):
        pass

    def build_system_message(self, content: str) -> Dict[str, str]:
        return {"role": "system", "content": content}


    def build_user_message(self, content: str) -> Dict[str, str]:
        return {"role": "user", "content": content}


    def build_prompt(self,
                     model: str,
                     system_prompt: str,
                     user_prompt: Optional[str] = None,
                     tools: Optional[List[Dict[str, str]]] = None
                     ) -> List[Dict[str, str]]:
        """
        Assemble tous les composants en une liste de messages structurée.
        """
        messages = []

        # Ajout du système
        if system_prompt:
            messages.append(self.build_system_message(system_prompt))

        # Introduction utilisateur
        if user_prompt:
            messages.append(self.build_user_message(user_prompt))


        prompt={}
        prompt["model"]=model
        prompt["messages"]=messages

        # Outils
        if tools:
            prompt["tools"]=tools

        # Message final
        #messages.append(self.build_user_message(new_user_message))

        return prompt

    def find_function(self, tools: List[Dict[str, Any]], name: str) -> Dict[str, Any]:
        print(tools)
        if tools["function"]["name"]=="nom_fonction":
            print("ok")
            print(tools["function"]["parameters"])

    def add_parameter(self,schema,function_name, name, type, description,required):
        if schema["function"]["name"]==function_name:
            schema["function"]["parameters"]["properties"][name] = {
                "type": type,
                "description": description
            }

            if required and name not in schema["function"]["parameters"]["required"]:
                schema["function"]["parameters"]["required"].append(name)

            return schema

    def create_tool_definition(self,
            function_name: str,
            description: str,
            parameters: dict,
            required_fields: list
    ) -> dict:
        return {
            "type": "function",
            "function": {
                "name": function_name,
                "description": description,
                "parameters":parameters
            }
        }

In [222]:
from typing import List, Dict, Optional
from typing import List, Dict, Any
import json

class Prompt():
    def __init__(self):
        pass

    def build_system_message(self, content: str) -> Dict[str, str]:
        return {"role": "system", "content": content}


    def build_user_message(self, content: str) -> Dict[str, str]:
        return {"role": "user", "content": content}


    def build_prompt(self,
                     model: str,
                     system_prompt: str,
                     user_prompt: Optional[str] = None,
                     tools: Optional[List[Dict[str, str]]] = None
                     ) -> List[Dict[str, str]]:
        """
        Assemble tous les composants en une liste de messages structurée.
        """
        messages = []

        # Ajout du système
        if system_prompt:
            messages.append(self.build_system_message(system_prompt))

        # Introduction utilisateur
        if user_prompt:
            messages.append(self.build_user_message(user_prompt))


        prompt={}
        prompt["model"]=model
        prompt["messages"]=messages

        # Outils
        if tools:
            prompt["tools"]=tools

        # Message final
        #messages.append(self.build_user_message(new_user_message))

        return prompt

    def find_function(self, tools: List[Dict[str, Any]], name: str) -> Dict[str, Any]:
        print(tools)
        if tools["function"]["name"]=="nom_fonction":
            print("ok")
            print(tools["function"]["parameters"])

    def add_parameter(self,schema,function_name, name, type, description,required):
        if schema["function"]["name"]==function_name:
            schema["function"]["parameters"]["properties"][name] = {
                "type": type,
                "description": description
            }

            if required and name not in schema["function"]["parameters"]["required"]:
                schema["function"]["parameters"]["required"].append(name)

            return schema

    def create_tool_definition(self,
                               function_name: str,
                               description: str,
                               parameters: dict,
                               required_fields: list
                               ) -> dict:
        return {
            "type": "function",
            "function": {
                "name": function_name,
                "description": description,
                "parameters":parameters
            }
        }

    def generate_prompt(self,model,system_prompt,user_prompt,fonctions,properties):
        tools=[]
        for fonction in fonctions:
            parametres = {"type": "object","properties": {},"required": []}
            schema=self.create_tool_definition(fonction,"test_description"+fonction,parametres,"les paramètres requis")
            for propriete in properties:
                schema = self.add_parameter(schema,fonction,propriete,"entier","description"+propriete,"true")

            tools.append(schema)
        prompt=self.build_prompt(model,system_prompt,user_prompt,tools)
        return prompt


fonctions=["timer","weather"]
proprietes=["propriete_1","propriete_2"]
prompt=Prompt().generate_prompt("gpt_4","tu es un assistant expert","quelle est la météo à liège",fonctions,proprietes)
print(json.dumps(prompt, indent=2, ensure_ascii=False))
print(prompt["model"])
print(prompt["messages"])
print(prompt["tools"])

{
  "model": "gpt_4",
  "messages": [
    {
      "role": "system",
      "content": "tu es un assistant expert"
    },
    {
      "role": "user",
      "content": "quelle est la météo à liège"
    }
  ],
  "tools": [
    {
      "type": "function",
      "function": {
        "name": "timer",
        "description": "test_descriptiontimer",
        "parameters": {
          "type": "object",
          "properties": {
            "propriete_1": {
              "type": "entier",
              "description": "descriptionpropriete_1"
            },
            "propriete_2": {
              "type": "entier",
              "description": "descriptionpropriete_2"
            }
          },
          "required": [
            "propriete_1",
            "propriete_2"
          ]
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "weather",
        "description": "test_descriptionweather",
        "parameters": {
          "type": "object",
          "

In [223]:
from prompt import Prompt
fonctions=["timer: il s'agit d'une fonction qui sert à déterminer l'heure actuelle","weather : i s'agit d'une fonction qui permet de déterminer la météo"]
proprietes=["propriete_1","propriete_2"]

prompt=Prompt().generate_prompt("gpt-4o","tu es un assistant expert","quelle est la météo à liège",fonctions,proprietes)
print(prompt["model"])

TypeError: 'NoneType' object is not subscriptable